In [1]:
#### !/usr/local/bin/env python
from __future__ import division
import numpy as np
from numpy import array
from numpy.random import sample as rs
from numpy import newaxis as na
import pandas as pd 
from scipy.stats import distributions
from scipy.stats import sem
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from radd.rl import multirace, analyzer
from scipy.stats import pearsonr


temporal_dynamics = lambda p, t: np.cosh(p['xb'][:, na] * t)
updateQ = lambda q, winner, r, A: q[winner][-1] + A*(r - q[winner][-1])
boltzmann_choiceP = lambda q, name, B: np.exp(B*q[name][-1])/np.sum([np.exp(B*q[k][-1]) for k in q.keys()])

homedir = os.path.expanduser('~')
cards = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/IGTob/IGTCards.csv"))
cards_scaled=cards.copy()*.01
cards_normed = (cards-cards.mean().mean())/cards.std().mean()

datadf = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/IGTob/IGT_data.csv"))
dat = datadf.dropna()
ob = dat.Obesity_score

In [2]:
ob_m = ob[(ob>=-1.25)&(ob<=1.25)]
ob_l = ob[ob<-1.25]
ob_h = ob[ob>1.25]
#ob_mhigh = ob[(ob<0)&(ob>-1)]
ob_dists = [ob_l, ob_m, ob_h]

In [6]:
for dist in ob_dists:
    sns.distplot(dist)

In [7]:

f, ax = plt.subplots(1, figsize=(5,5))
vs = dat.Cards_RNR_VS_mean#[dat.Cards_RNR_VS_mean>.1]
pay = dat.P_payoff
vs_dists = []
p_dists = []
#ob_dists = [ob_low, ob_mlow, ob_mhigh, ob_high]
#ob_dists = [ob_mlow, ob_mhigh]
for dist in ob_dists:
    p_dists.append(pay.loc[dist.index])
    vs_dists.append(vs.loc[dist.index])

labels=['low', 'med', 'high']    
#dist.iloc
for i in range(len(ob_dists)):
    print(pearsonr(p_dists[i].values, vs_dists[i].values))
    sns.regplot(p_dists[i], vs_dists[i], ax=ax, label=labels[i])
plt.legend(loc=0)
sns.despine()

#plt.savefig('hi_m_low_obesity_vspay_rho.png', dpi=300)

(0.21633625044648147, 0.22657766759593848)
(0.14580110179854588, 0.017767560154263218)
(-0.12409931295410093, 0.53742109541887761)


In [9]:
ob = dat.Obesity_score
pay = dat.P_payoff.values
sens = dat.Q_sensitivity.values
xdf = pd.DataFrame([ob, pay, sens]).T#], columns=['O', 'P', 'Q'])
#ob.dropna
#ob[np.isnan(ob)]


high_vs  = dat.Cards_RNR_VS_mean#[dat.Cards_RNR_VS_mean>.1]
highvs_q = dat.loc[high_vs.index.values, 'Q_sensitivity']
highvs_p = dat.loc[high_vs.index.values, 'P_payoff']
highvs_ob = dat.loc[high_vs.index.values, 'Obesity_score']

sns.regplot(highvs_ob, highvs_q)
print(pearsonr(highvs_ob, highvs_q))

sns.regplot(highvs_ob, highvs_p)
print(pearsonr(highvs_ob, highvs_p))

(-0.11825057914705488, 0.033357430063340701)
(-0.14188078995644088, 0.010560003042315965)


In [33]:
p={'vd':.7, 'vi':.4}; p['a']=.4; p['tr']=.2; p['xb']=1.5
#pdict = theta.random_inits(['vd', 'vi', 'tr', 'a', 'xb'])
#p = deepcopy(pdict)
output = multirace.run_trials(p, cards_scaled.copy(), nblocks=2, a_go=.3, a_no=.05, beta=0.2)
choices, rts, all_traces, qdict,choice_prob, vdhist, vihist = output
Pi,Qi = analyzer.igt_scores(np.asarray(choices))

In [34]:
from radd.rl import visr
visr.plot_summary(output)